# Jupyter Notebook per l'execució del codi cel·la a cel·la 

## Imports

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import time
import random
from datetime import datetime
from multiprocessing import Pool, cpu_count
from rdkit import Chem
from rdkit.Chem import Draw, DataStructs
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
from mordred import Calculator, descriptors
from mordred.error import MissingValueBase
from scipy.optimize import linear_sum_assignment
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error
import cupy as cp
import itertools


## Cel·la 1: Càlcul de la GED i distàncies Mordred a la base de dades ESOL.csv mitjançant CPU

In [ ]:
# Configuración inicial
archivo_csv = "ESOL.csv"
columnas_seleccionadas = ["Compound ID", "ESOL predicted log solubility in mols per litre", "smiles"]

print("Inici execucio")

# Leer y procesar datos
df_procesado = pd.read_csv(archivo_csv)[columnas_seleccionadas]

# Estructuras para almacenamiento
grafosQuimicos = []
grafosDefault = []
descriptores_mordred = {}

# Inicializar calculador Mordred
calc = Calculator(descriptors, ignore_3D=True)

# Procesar moléculas
for index, row in df_procesado.iterrows():
    smiles = row["smiles"]
    compound_id = row["Compound ID"]
    solubility = row["ESOL predicted log solubility in mols per litre"]
    
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is not None:
        try:
            # Calcular descriptores Mordred
            desc = calc(mol)
            desc = desc.fill_missing(0)  # Sustituir valores faltantes por 0
            descriptores_mordred[compound_id] = np.array([float(d) for d in desc])
        except MissingValueBase as e:
            print(f"Error en descriptores para {compound_id}: {e}")
            continue

        # Crear grafos
        G = nx.Graph()
        for atom in mol.GetAtoms():
            G.add_node(atom.GetIdx(), element=atom.GetSymbol())
        for bond in mol.GetBonds():
            G.add_edge(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx(), bond_type=bond.GetBondType())
        
        grafosQuimicos.append((compound_id, G))

        # Grafo simplificado para GED
        G2 = nx.Graph()
        G2.add_nodes_from(G.nodes)
        G2.add_edges_from(G.edges)
        grafosDefault.append((compound_id, G2))

# Función para calcular GED y escribir en CSV
def calcular_ged_y_escribir(pair):
    id1, g1, id2, g2 = pair
    ged = nx.graph_edit_distance(g2, g1, timeout=300)
    return (id1, id2, ged)

# Función para calcular distancia Mordred y escribir en CSV
def calcular_distancia_mordred_y_escribir(pair):
    id1, id2 = pair
    dist = np.linalg.norm(descriptores_mordred[id1] - descriptores_mordred[id2])
    return (id1, id2, dist)

# Calcular GED en paralelo y escribir secuencialmente
start_time = time.time()

# Preparar pares de grafos para GED
ged_pairs = [(id1, g1, id2, g2) for (id1, g1), (id2, g2) in itertools.combinations(grafosDefault, 2)]

# Abrir el archivo CSV en modo de escritura
with open('ged_results.csv', mode='w', newline='') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(['id1', 'id2', 'ged'])  # Escribir la cabecera

    # Calcular GED en paralelo y escribir secuencialmente
    with Pool(cpu_count()) as pool:
        for result in pool.imap(calcular_ged_y_escribir, ged_pairs):
            csv_writer.writerow(result)  # Escribir cada resultado en el archivo CSV

# Calcular distancias Mordred en paralelo y escribir secuencialmente
mordred_pairs = list(itertools.combinations(descriptores_mordred.keys(), 2))

# Abrir el archivo CSV en modo de escritura
with open('distancias_mordred.csv', mode='w', newline='') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(['id1', 'id2', 'distancia_mordred'])  # Escribir la cabecera

    # Calcular distancias Mordred en paralelo y escribir secuencialmente
    with Pool(cpu_count()) as pool:
        for result in pool.imap(calcular_distancia_mordred_y_escribir, mordred_pairs):
            csv_writer.writerow(result)  # Escribir cada resultado en el archivo CSV

# Resultados finales
print("\nResumen de resultados:")
print(f"- Tiempo total de ejecución: {time.time() - start_time:.2f} segundos")

# Mostrar gráficos
plt.show()

## Cel·la 2: Càlcul de la GED a la base de dades ESOL.csv mitjançant GPU

In [ ]:
#Valores asignados a substituir, insertar o eliminar nodos o arestas
node_sub_cost = 2
node_del_cost = 1
node_ins_cost = 1

# Example cost functions
def node_sub_cost(u, v):
    return 2

def node_del_cost(u):
    return 1

def node_ins_cost(v):
    return 1

def fast_bipartite_distance_gpu(pair):

    id1, s1, Gp, id2, s2, Gq = pair

    nodes_p = list(Gp.nodes())
    nodes_q = list(Gq.nodes())
    n, m = len(nodes_p), len(nodes_q)

    D_gpu = cp.array([node_del_cost(u) for u in nodes_p])
    I_gpu = cp.array([node_ins_cost(v) for v in nodes_q])
    C_vedi = float(D_gpu.sum().get() + I_gpu.sum().get())

    cost_matrix_gpu = cp.zeros((n, m), dtype=cp.float32)
    for i, u in enumerate(nodes_p):
        for j, v in enumerate(nodes_q):
            sub_cost = node_sub_cost(u, v)
            cost_matrix_gpu[i, j] = sub_cost - (D_gpu[i] + I_gpu[j])

    cost_matrix_cpu = cp.asnumpy(cost_matrix_gpu)
    row_ind, col_ind = linear_sum_assignment(cost_matrix_cpu)
    edit_cost0 = cost_matrix_cpu[row_ind, col_ind].sum()

    return (id1, s1, id2, s2, edit_cost0 + C_vedi)

# Configuración inicial
archivo_csv = "ESOL.csv"
columnas_seleccionadas = ["Compound ID", "smiles"]

# Leer y procesar datos
df_procesado = pd.read_csv(archivo_csv)[columnas_seleccionadas]

# Estructuras para almacenamiento
grafos_para_ged = []

# Procesar moléculas
for index, row in df_procesado.iterrows():
    smiles = row["smiles"]
    compound_id = row["Compound ID"]
    
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is not None:
        # Crear grafo NetworkX
        G = nx.Graph()
        for atom in mol.GetAtoms():
            G.add_node(atom.GetIdx(), atom_type=atom.GetSymbol())
        for bond in mol.GetBonds():
            G.add_edge(bond.GetBeginAtomIdx(), 
                      bond.GetEndAtomIdx(), 
                      bond_type=bond.GetBondType())
        
        grafos_para_ged.append((compound_id, smiles, G))

if __name__ == '__main__':

    # Calcular GED en paralelo usando GPU
    start_time = time.time()

    # Preparar pares de grafos para GED (usando solo los dos primeros grafos para prueba)
    ged_pairs = [(id1, s1, g1, id2, s2, g2) 
                for (id1, s1, g1), (id2, s2, g2) in itertools.combinations(grafos_para_ged, 2)]

    # Escribir resultados en CSV
    with open('ged_gpu_ESOL.csv', mode='w', newline='') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(['id1', 'smiles1', 'id2', 'smiles2', 'ged'])
        
        # Usar solo 1 proceso ya que la GPU maneja el paralelismo interno
        with Pool(1) as pool:  # Reducir procesos para evitar saturación de GPU
            for result in pool.imap(fast_bipartite_distance_gpu, ged_pairs):
                csv_writer.writerow(result)
                file.flush()  # Asegurar escritura inmediata

    # Resultados finales
    print("\nResumen de resultados con GPU optimizado:")
    print(f"- Tiempo total de ejecución: {time.time() - start_time:.2f} segundos")
    print(f"- Número de comparaciones realizadas: {len(ged_pairs)}")


## Cel·la 3: Càlcul de la GED i distàncies Mordred a la base de dades FreeSolv.csv mitjançant CPU

In [ ]:
# Configuración inicial
archivo_csv = "FreeSolv.csv"
columnas_seleccionadas = ["iupac", "smiles"]

print("Inici execucio")

# Leer y procesar datos
df_procesado = pd.read_csv(archivo_csv)[columnas_seleccionadas]

# Estructuras para almacenamiento
grafosDefault = []
descriptores_mordred = {}

# Inicializar calculador Mordred
calc = Calculator(descriptors, ignore_3D=True)

# Procesar moléculas
for index, row in df_procesado.iterrows():
    smiles = row["smiles"]
    compound_id = row["iupac"]
    
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is not None:
        try:
            # Calcular descriptores Mordred
            desc = calc(mol)
            desc = desc.fill_missing(0)  # Sustituir valores faltantes por 0
            descriptores_mordred[compound_id] = np.array([float(d) for d in desc])
        except MissingValueBase as e:
            print(f"Error en descriptores para {compound_id}: {e}")
            continue

        # Crear grafos
        G = nx.Graph()
        for atom in mol.GetAtoms():
            G.add_node(atom.GetIdx(), element=atom.GetSymbol())
        for bond in mol.GetBonds():
            G.add_edge(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx(), bond_type=bond.GetBondType())
        
        # Grafo simplificado para GED
        G2 = nx.Graph()
        G2.add_nodes_from(G.nodes)
        G2.add_edges_from(G.edges)
        grafosDefault.append((compound_id, smiles, G2))

# Función para calcular GED y escribir en CSV
def calcular_ged_y_escribir(pair):
    id1, s1, g1, id2, s2, g2 = pair
    ged = nx.graph_edit_distance(g2, g1, timeout=300)
    return (id1, s1, id2, s2, ged)

# Función para calcular distancia Mordred y escribir en CSV
def calcular_distancia_mordred_y_escribir(pair):
    id1, id2 = pair
    dist = np.linalg.norm(descriptores_mordred[id1] - descriptores_mordred[id2])
    return (id1, id2, dist)

# Calcular GED en paralelo y escribir secuencialmente
start_time = time.time()

# Preparar pares de grafos para GED
ged_pairs = [(id1, s1, g1, id2, s2, g2) 
                for (id1, s1, g1), (id2, s2, g2) in itertools.combinations(grafosDefault, 2)]

# Abrir el archivo CSV en modo de escritura
with open('ged_cpu_Freesolv.csv', mode='w', newline='') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(['id1', 'smiles1', 'id2', 'smiles2', 'ged'])  # Escribir la cabecera

    # Calcular GED en paralelo y escribir secuencialmente
    with Pool(cpu_count()) as pool:
        for result in pool.imap(calcular_ged_y_escribir, ged_pairs):
            csv_writer.writerow(result)  # Escribir cada resultado en el archivo CSV

# Calcular distancias Mordred en paralelo y escribir secuencialmente
mordred_pairs = list(itertools.combinations(descriptores_mordred.keys(), 2))

# Abrir el archivo CSV en modo de escritura
with open('distancias_mordred_freesolv.csv', mode='w', newline='') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(['id1', 'id2', 'distancia_mordred'])  # Escribir la cabecera

    # Calcular distancias Mordred en paralelo y escribir secuencialmente
    with Pool(cpu_count()) as pool:
        for result in pool.imap(calcular_distancia_mordred_y_escribir, mordred_pairs):
            csv_writer.writerow(result)  # Escribir cada resultado en el archivo CSV

# Resultados finales
    print("\nResumen de resultados con CPU:")
    print(f"- Tiempo total de ejecución: {time.time() - start_time:.2f} segundos")
    print(f"- Número de comparaciones realizadas: {len(ged_pairs)}")
    
# Mostrar gráficos
plt.show()

## Cel·la 4: Càlcul de la GED a la base de dades FreeSolv.csv mitjançant GPU


In [ ]:
#Valores asignados a substituir, insertar o eliminar nodos o arestas
node_sub_cost = 2
node_del_cost = 1
node_ins_cost = 1

# Example cost functions
def node_sub_cost(u, v):
    return 2

def node_del_cost(u):
    return 1

def node_ins_cost(v):
    return 1

def fast_bipartite_distance_gpu(pair):

    id1, s1, Gp, id2, s2, Gq = pair

    nodes_p = list(Gp.nodes())
    nodes_q = list(Gq.nodes())
    n, m = len(nodes_p), len(nodes_q)

    D_gpu = cp.array([node_del_cost(u) for u in nodes_p])
    I_gpu = cp.array([node_ins_cost(v) for v in nodes_q])
    C_vedi = float(D_gpu.sum().get() + I_gpu.sum().get())

    cost_matrix_gpu = cp.zeros((n, m), dtype=cp.float32)
    for i, u in enumerate(nodes_p):
        for j, v in enumerate(nodes_q):
            sub_cost = node_sub_cost(u, v)
            cost_matrix_gpu[i, j] = sub_cost - (D_gpu[i] + I_gpu[j])

    cost_matrix_cpu = cp.asnumpy(cost_matrix_gpu)
    row_ind, col_ind = linear_sum_assignment(cost_matrix_cpu)
    edit_cost0 = cost_matrix_cpu[row_ind, col_ind].sum()

    return (id1, s1, id2, s2, edit_cost0 + C_vedi)

# Configuración inicial
archivo_csv = "FreeSolv.csv"
columnas_seleccionadas = ["iupac", "smiles"]

# Leer y procesar datos
df_procesado = pd.read_csv(archivo_csv)[columnas_seleccionadas]

# Estructuras para almacenamiento
grafos_para_ged = []

# Procesar moléculas
for index, row in df_procesado.iterrows():
    smiles = row["smiles"]
    compound_id = row["iupac"]
    
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is not None:
        # Crear grafo NetworkX
        G = nx.Graph()
        for atom in mol.GetAtoms():
            G.add_node(atom.GetIdx(), atom_type=atom.GetSymbol())
        for bond in mol.GetBonds():
            G.add_edge(bond.GetBeginAtomIdx(), 
                      bond.GetEndAtomIdx(), 
                      bond_type=bond.GetBondType())
        
        grafos_para_ged.append((compound_id, smiles, G))

if __name__ == '__main__':

    # Calcular GED en paralelo usando GPU
    start_time = time.time()

    # Preparar pares de grafos para GED (usando solo los dos primeros grafos para prueba)
    ged_pairs = [(id1, s1, g1, id2, s2, g2) 
                for (id1, s1, g1), (id2, s2, g2) in itertools.combinations(grafos_para_ged, 2)]

    # Escribir resultados en CSV
    with open('ged_results_gpu_optimized.csv', mode='w', newline='') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(['id1', 'smiles1', 'id2', 'smiles2', 'ged'])
        
        # Usar solo 1 proceso ya que la GPU maneja el paralelismo interno
        with Pool(1) as pool:  # Reducir procesos para evitar saturación de GPU
            for result in pool.imap(fast_bipartite_distance_gpu, ged_pairs):
                csv_writer.writerow(result)
                file.flush()  # Asegurar escritura inmediata

    # Resultados finales
    print("\nResumen de resultados con GPU optimizado:")
    print(f"- Tiempo total de ejecución: {time.time() - start_time:.2f} segundos")
    print(f"- Número de comparaciones realizadas: {len(ged_pairs)}")


## Cel·la 5: Implementació de la KNN i visualització de mètriques i gràfics per GED 

In [ ]:
# Función para imprimir con timestamp
def print_ts(msg):
    print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}")

# Cargar datos
df = pd.read_csv("ged_cpu_Freesolv.csv")    # Cambiar por dataset deseado
df.columns = ['id1', 'smiles1', 'id2', 'smiles2', 'ged']

# Crear generador de Morgan fingerprint (nuevo método)
print_ts("Crear generador de Morgan fingerprint (nuevo método)")
generator = GetMorganGenerator(radius=2, fpSize=2048)

# Función para obtener fingerprint con el nuevo enfoque
def get_morgan_fp(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    fp = generator.GetFingerprint(mol)
    arr = np.zeros((fp.GetNumBits(),), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

# Calcular fingerprints
print_ts("Calcular fingerprints")
df['fp1'] = df['smiles1'].apply(get_morgan_fp)
df['fp2'] = df['smiles2'].apply(get_morgan_fp)

# Eliminar filas inválidas
print_ts("Eliminar filas inválidas")
df = df.dropna(subset=['fp1', 'fp2'])

# Crear vectores de diferencia absoluta
print_ts("Crear vectores de diferencia absoluta")
X = [np.abs(f1 - f2) for f1, f2 in zip(df['fp1'], df['fp2'])]
y = df['ged'].values

# Usamos indices para trackear la división
indices = df.index.to_list()
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, indices, test_size=0.3, random_state=42
)

# Train-test split (70% entrenamiento, 30% test)
print_ts("Train-test split (70% entrenamiento, 30% test)")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Búsqueda del mejor valor de K con GridSearchCV (usando MSE)
print_ts("Búsqueda del mejor valor de K con GridSearchCV (usando MSE)")
param_grid = {'n_neighbors': list(range(1, 21))}
knn = KNeighborsRegressor()
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

best_k = grid_search.best_params_['n_neighbors']
print_ts(f"Mejor valor de K encontrado: {best_k}")

# Entrenar modelo con el mejor K
print_ts("Entrenar modelo con el mejor K")
model = KNeighborsRegressor(n_neighbors=best_k)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Métricas de evaluación
print_ts("Cálculo métricas de evaluación")
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nResultados del modelo KNN (K={best_k}):")
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")

# Crear nuevo DataFrame con las filas del conjunto de test
df_test = df.loc[idx_test].copy()
df_test['ged_predicha'] = y_pred
df_test['error_absoluto'] = np.abs(df_test['ged'] - df_test['ged_predicha'])

# Guardar a CSV
df_test.to_csv("TestErrors.csv", index=False)

# Gráfico 1: GED real vs predicho
plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--')
plt.xlabel('GED real')
plt.ylabel('GED predicha')
plt.title('GED real vs GED predicha')
plt.grid(True)
plt.tight_layout()
plt.show(block=False)

# Gráfico 2: MSE vs K
print_ts("Graficar MSE vs K")
mse_scores = []
k_range = list(range(1, 21))

for k in k_range:
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred_k = knn.predict(X_test)
    mse_k = mean_squared_error(y_test, y_pred_k)
    mse_scores.append(mse_k)

print_ts("Fin ejecución")
plt.figure(figsize=(8, 6))
plt.plot(k_range, mse_scores, marker='o', color='orange')
plt.xlabel('Número de vecinos (K)')
plt.ylabel('MSE')
plt.title('Curva de error MSE vs K')
plt.xticks(k_range)
plt.grid(True)
plt.tight_layout()
plt.show(block=False)



# Elegir una muestra aleatoria de 10.000 predicciones
sample_size = 10000
indices = random.sample(range(len(y_test)), min(sample_size, len(y_test)))
y_test_sample = [y_test[i] for i in indices]
y_pred_sample = [y_pred[i] for i in indices]

plt.figure(figsize=(8,6))
plt.scatter(y_test_sample, y_pred_sample, alpha=0.3, s=5)
plt.plot([min(y_test_sample), max(y_test_sample)], [min(y_test_sample), max(y_test_sample)], 'r--')
plt.xlabel('GED real')
plt.ylabel('GED predicha')
plt.title('GED real vs predicha (muestra de 10.000)')
plt.grid(True)
plt.tight_layout()
plt.show(block=False)

# Hexbin Plot (gráfico de densidad 2D)
plt.figure(figsize=(8,6))
plt.hexbin(y_test, y_pred, gridsize=100, cmap='Blues', bins='log')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--')
plt.xlabel('GED real')
plt.ylabel('GED predicha')
plt.title('Densidad GED real vs predicha (hexbin)')
plt.colorbar(label='log(N° de puntos)')
plt.tight_layout()
plt.show(block=False)

# Gráfico de residuales agregados por bins

residuals = np.array(y_test) - np.array(y_pred)
df_res = pd.DataFrame({'GED real': y_test, 'Residual': residuals})

plt.figure(figsize=(8,6))
sns.regplot(data=df_res, x='GED real', y='Residual', scatter_kws={'alpha':0.1, 's':5}, lowess=True)
plt.axhline(0, color='red', linestyle='--')
plt.title('Gráfico de residuales vs GED real')
plt.grid(True)
plt.tight_layout()
plt.show()


## Cel·la 6: Implementació de la KNN i visualització de mètriques i gràfics per Mordred 

In [ ]:
# Función para imprimir con timestamp
def print_ts(msg):
    print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}")

# Cargar datos
df = pd.read_csv("mordred_ESOL.csv")    # Cambiar por dataset deseado
df.columns = ['id1', 'smiles1', 'id2', 'smiles2', 'distancia_mordred']

# Crear generador de Morgan fingerprint (nuevo método)
print_ts("Crear generador de Morgan fingerprint (nuevo método)")
generator = GetMorganGenerator(radius=2, fpSize=2048)

# Función para obtener fingerprint con el nuevo enfoque
def get_morgan_fp(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    fp = generator.GetFingerprint(mol)
    arr = np.zeros((fp.GetNumBits(),), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

# Calcular fingerprints
print_ts("Calcular fingerprints")
df['fp1'] = df['smiles1'].apply(get_morgan_fp)
df['fp2'] = df['smiles2'].apply(get_morgan_fp)

# Eliminar filas inválidas
print_ts("Eliminar filas inválidas")
df = df.dropna(subset=['fp1', 'fp2'])

# Crear vectores de diferencia absoluta
print_ts("Crear vectores de diferencia absoluta")
X = [np.abs(f1 - f2) for f1, f2 in zip(df['fp1'], df['fp2'])]
y = df['distancia_mordred'].values

# Usamos indices para trackear la división
indices = df.index.to_list()
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, indices, test_size=0.3, random_state=42
)

# Train-test split (70% entrenamiento, 30% test)
print_ts("Train-test split (70% entrenamiento, 30% test)")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Búsqueda del mejor valor de K con GridSearchCV (usando MSE)
print_ts("Búsqueda del mejor valor de K con GridSearchCV (usando MSE)")
param_grid = {'n_neighbors': list(range(1, 21))}
knn = KNeighborsRegressor()
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

best_k = grid_search.best_params_['n_neighbors']
print_ts(f"Mejor valor de K encontrado: {best_k}")

# Entrenar modelo con el mejor K
print_ts("Entrenar modelo con el mejor K")
model = KNeighborsRegressor(n_neighbors=best_k)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Métricas de evaluación
print_ts("Cálculo métricas de evaluación")
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nResultados del modelo KNN (K={best_k}):")
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")

# Crear nuevo DataFrame con las filas del conjunto de test
df_test = df.loc[idx_test].copy()
df_test['distancia_mordred_predicha'] = y_pred
df_test['error_absoluto'] = np.abs(df_test['distancia_mordred'] - df_test['distancia_mordred_predicha'])

# Guardar a CSV
df_test.to_csv("TestErrors.csv", index=False)

# Gráfico 1: GED real vs predicho
plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--')
plt.xlabel('Distancia Mordred real')
plt.ylabel('Distancia Mordred predicha')
plt.title('Distancia Mordred real vs Distancia Mordred predicha')
plt.grid(True)
plt.tight_layout()
plt.show(block=False)

# Gráfico 2: MSE vs K
print_ts("Graficar MSE vs K")
mse_scores = []
k_range = list(range(1, 21))

for k in k_range:
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred_k = knn.predict(X_test)
    mse_k = mean_squared_error(y_test, y_pred_k)
    mse_scores.append(mse_k)

print_ts("Fin ejecución")
plt.figure(figsize=(8, 6))
plt.plot(k_range, mse_scores, marker='o', color='orange')
plt.xlabel('Número de vecinos (K)')
plt.ylabel('MSE')
plt.title('Curva de error MSE vs K')
plt.xticks(k_range)
plt.grid(True)
plt.tight_layout()
plt.show(block=False)



# Elegir una muestra aleatoria de 10.000 predicciones
sample_size = 10000
indices = random.sample(range(len(y_test)), min(sample_size, len(y_test)))
y_test_sample = [y_test[i] for i in indices]
y_pred_sample = [y_pred[i] for i in indices]

plt.figure(figsize=(8,6))
plt.scatter(y_test_sample, y_pred_sample, alpha=0.3, s=5)
plt.plot([min(y_test_sample), max(y_test_sample)], [min(y_test_sample), max(y_test_sample)], 'r--')
plt.xlabel('Distancia Mordred real')
plt.ylabel('Distancia Mordred predicha')
plt.title('Distancia Mordred real vs predicha (muestra de 10.000)')
plt.grid(True)
plt.tight_layout()
plt.show(block=False)

# Hexbin Plot (gráfico de densidad 2D)
plt.figure(figsize=(8,6))
plt.hexbin(y_test, y_pred, gridsize=100, cmap='Blues', bins='log')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--')
plt.xlabel('Distancia Mordred real')
plt.ylabel('Distancia Mordred predicha')
plt.title('Densidad Distancia Mordred real vs predicha (hexbin)')
plt.colorbar(label='log(N° de puntos)')
plt.tight_layout()
plt.show(block=False)

# Gráfico de residuales agregados por bins

residuals = np.array(y_test) - np.array(y_pred)
df_res = pd.DataFrame({'Distancia Mordred real': y_test, 'Residual': residuals})

plt.figure(figsize=(8,6))
sns.regplot(data=df_res, x='Distancia Mordred real', y='Residual', scatter_kws={'alpha':0.1, 's':5}, lowess=True)
plt.axhline(0, color='red', linestyle='--')
plt.title('Gráfico de residuales vs Distancia Mordred real')
plt.grid(True)
plt.tight_layout()
plt.show()


## Cel·la 7: EXTRA: Comparativa entre approach GPU i CPU

In [ ]:



def mostrar_molec(df, n=3):
    imgs = []
    for i in range(n):
        row = df.iloc[i]
        mol1 = Chem.MolFromSmiles(row["smiles1"])
        mol2 = Chem.MolFromSmiles(row["smiles2"])
        mols = [mol1, mol2]
        legends = [f"{row['id1']}", f"{row['id2']}"]
        img = Draw.MolsToImage(mols, legends=legends, subImgSize=(300,300))
        imgs.append(img)
    
    for img in imgs:
        img.show()

#Carreguem arxius
cpu_df = pd.read_csv("ged_cpu_Freesolv.csv")
gpu_df = pd.read_csv("ged_gpu_Freesolv.csv")

#Merge dels datasets
merged = pd.merge(cpu_df, gpu_df, on=["id1", "smiles1", "id2", "smiles2"], suffixes=('_cpu', '_gpu'))
merged["ged_diff"] = merged["ged_cpu"]-merged["ged_gpu"]
merged["ged_abs_diff"] = merged["ged_diff"].abs()
merged["ged_rel_diff"] = (merged["ged_abs_diff"])/(merged[["ged_cpu", "ged_gpu"]].max(axis=1))
print(merged.head(10))
#Càlcul metriques errors
mae = mean_absolute_error(merged["ged_cpu"], merged["ged_gpu"])
rmse = root_mean_squared_error(merged["ged_cpu"], merged["ged_gpu"])
r2 = r2_score(merged["ged_cpu"], merged["ged_gpu"])
n = len(merged)

mostrar_molec(merged.head(10))

print(f"MAE (Mean average error): {mae:.4f}")
print(f"RMSE (Root Mean Squared Error): {rmse:.4f}")
print(f"R^2: {r2:.4f}")

#Filtre de GED <= 5 
filter = merged[merged['ged_abs_diff']<6]
print(f"\nFiltered amb GED<=5: {len(filter)}")
#filter.to_csv("ged_ESOL_diff_menor6.csv")
print(filter.head(10))

#Visualització Distribució
plt.figure(figsize=(10,5))
sns.histplot(merged["ged_diff"], bins=50, kde=True)
plt.title("Distribució Diferències (CPU - GPU)")
plt.xlabel("Diferència GED")
plt.ylabel("Freqüència")
plt.show(block=False)

#Visualització del scatterplot
x = merged['ged_cpu']
y = merged['ged_gpu']

slope, intercept = np.polyfit(x,y,1)
y_pred = slope * x + intercept
r2graph = r2_score(y, y_pred)

plt.figure(figsize=(8,8))
sns.scatterplot(x=x, y=y, alpha=0.5, label='Dades')
plt.plot(x, y_pred, color='red', label='Regressió', linewidth=2)

#Linia identitat
min_val, max_val = min(x.min(), y.min()), max(x.max(), y.max())
plt.plot([min_val, max_val], [min_val,max_val], color='blue', linestyle='--', label='Identitat')
eq_text = f'y = {slope:.3f}x + {intercept:.3f}\nR² = {r2graph:.4f}\nN = {n}'
plt.text(min_val, max_val, eq_text, fontsize=12, ha='left', va='top', bbox=dict(facecolor='white', alpha=0.7))
plt.xlabel("GED (CPU)")
plt.ylabel("GED (GPU)")
plt.title("Comparativa GED CPU vs GPU")
plt.grid(True)
plt.tight_layout()
plt.show()
